## Which skills and job features have the biggest impact on salary?

Here we'll uncover factors that influence the salary range for Data Analysts and will specify:
- Must-have skills for Data Analysts and how they affect salary
- Top-paying analytical skills and their impact on salary
- Impact of job type (remote vs on-site) and degree requirements on salary
- Key drivers of Data Analyst salaries (SHAP analysis)

### Import Libraries

In [1]:
import ast
from pathlib import Path

import numpy as np
import pandas as pd

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MultiLabelBinarizer
from xgboost import XGBRegressor
import shap

import warnings
warnings.filterwarnings("ignore", category=UserWarning)

from sklearn.metrics import mean_squared_error, r2_score

from statsmodels.tsa.statespace.sarimax import SARIMAX


### Load Cleaned Dataset

In [2]:
df = pd.read_pickle(Path.cwd().parents[1] / 'Raw_Data' / 'df_Final_2.pkl')

### Remove Salary Outliers
- Outliers were removed per job title and country to preserve meaningful comparisons.
- For countries with fewer than 5 postings, outliers were removed based on job title only to avoid skewed insights from limited data.

In [3]:
#Calculate global IQR
Q1_global = df['salary_month_avg_eur'].quantile(0.25)
Q3_global = df['salary_month_avg_eur'].quantile(0.75)
IQR_global = Q3_global - Q1_global
lower_bound_global = Q1_global - 1.5 * IQR_global
upper_bound_global = Q3_global + 1.5 * IQR_global

filtered_groups = []

#Iterate over groups manually
for (country, title), group in df.groupby(['job_country', 'job_title_short']):
    if len(group) >= 10:
        Q1 = group['salary_month_avg_eur'].quantile(0.25)
        Q3 = group['salary_month_avg_eur'].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR
    else:
        lower = lower_bound_global
        upper = upper_bound_global

    filtered = group[
        (group['salary_month_avg_eur'] >= lower) &
        (group['salary_month_avg_eur'] <= upper)
    ]
    filtered_groups.append(filtered)

# Combine all groups back
df_filtered = pd.concat(filtered_groups, ignore_index=True)

print(f'Original dataset size: {len(df)}')
print(f'Filtered dataset size: {len(df_filtered)}')

Original dataset size: 42335
Filtered dataset size: 41036


## Predicting Data Scientist Salaries Based on Analyst Transition Skills

To support data analysts transitioning to data science roles, we build a regression model to estimate salary differences based on key overlapping skills:

**Target Role:** Data Scientist  
**Key Transition Skills:** `Python`, `SQL`, `Tableau`  

We'll filter Data Scientist job postings, create binary skill features, and predict monthly salary in EUR.

In [9]:
df_ml = df_filtered[df_filtered['job_title_short'] == 'Data Scientist'].copy()

df_filtered = df_filtered[df_filtered['job_skills'].notna()].copy()

print(f"Valid Data Scientist rows: {len(df_ml)}")

Valid Data Scientist rows: 9564


In [10]:
# Define target skills
transition_skills = ['python', 'sql', 'tableau', 'aws', 'spark', 'r', 'excel']

# Add binary columns for skill presence
for skill in transition_skills:
    df_ml[f'has_{skill}'] = df_ml['job_skills'].apply(
        lambda skills: skill in [s.lower() for s in skills]
    )

# Add other static/binary features
df_ml['job_schedule_full_time'] = (df_ml['job_schedule_type'] == 'Full-time').astype(int)

# Check result
print(df_ml[[f'has_{s}' for s in transition_skills] + 
            ['job_schedule_full_time']].sum())

has_python                7810
has_sql                   5774
has_tableau               2187
has_aws                   1869
has_spark                 1778
has_r                     4414
has_excel                 1057
job_schedule_full_time    8613
dtype: int64


In [ ]:
feature_cols = [
    'has_python', 'has_sql', 'has_tableau', 'has_aws', 'has_spark', 'has_r', 'has_excel', 'job_schedule_full_time']

X = df_ml[feature_cols].astype(int)  # Ensure all are numeric
y = df_ml['salary_month_avg_eur']

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Train linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict and evaluate
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred) ** 0.5

print("R² Score:", r2_score(y_test, y_pred))
print("RMSE (EUR/month):", round(rmse, 2))

R² Score: 0.0442313531053653
RMSE (EUR/month): 3292.9


A simple linear model was trained to predict Data Scientist salaries based on the presence of three common skills shared with Data Analyst roles: Python, SQL, and Tableau.
While these are foundational tools, the model showed limited predictive power:

R² Score: ~0.025 → These three skills explain only ~2.5% of salary variation
RMSE: ~€4,124/month → Predictions deviate significantly from actual salaries
🔎 Interpretation:
Although Python, SQL, and Tableau are essential for transitioning into Data Science, they are not sufficient alone to predict salary. Other factors — such as experience level, cloud or ML-specific skills, and company region or size — likely play a much larger role.
🧭 Takeaway:
Job seekers should view these shared skills as a starting point, and plan to develop more specialized competencies (e.g. machine learning, cloud platforms, deep learning frameworks) to maximize salary growth in Data Scientist roles.

In [ ]:
# Prepare monthly time series for Data Scientist salary
eu_us = ['EU', 'US']
df_ds = df_filtered[
    (df_filtered['region_group'].isin(eu_us)) &
    (df_filtered['job_title_short'] == 'Data Scientist')
].copy()
df_ds['job_posted_date'] = pd.to_datetime(df_ds['job_posted_date'])
df_ds['year'] = df_ds['job_posted_date'].dt.year
df_ds['month'] = df_ds['job_posted_date'].dt.month

# Group by year and month for salary (median)
monthly_stats = (
    df_ds.groupby(['year', 'month'])
    .agg(median_salary=('salary_month_avg_eur', 'median'))
    .reset_index()
)

# Create a datetime index for time series
monthly_stats['date'] = pd.to_datetime(monthly_stats[['year', 'month']].assign(day=1))
monthly_stats = monthly_stats.sort_values('date').set_index('date')

# Fill missing months (if any)
all_months = pd.date_range(monthly_stats.index.min(), monthly_stats.index.max(), freq='MS')
monthly_stats = monthly_stats.reindex(all_months)
monthly_stats['median_salary'] = monthly_stats['median_salary'].interpolate()

# --- SARIMA for Salary (Median) ---
sarima_salary = SARIMAX(
    monthly_stats['median_salary'],
    order=(1,1,1),
    seasonal_order=(1,1,1,12),
    enforce_stationarity=False,
    enforce_invertibility=False
).fit(disp=False)

# Forecast for 2025 (12 months ahead)
n_forecast = 12
future_dates = pd.date_range(monthly_stats.index[-1] + pd.offsets.MonthBegin(1), periods=n_forecast, freq='MS')
salary_forecast = sarima_salary.get_forecast(steps=n_forecast)
salary_pred = salary_forecast.predicted_mean
salary_ci = salary_forecast.conf_int()

# --- Trendline (linear regression) ---
X = np.arange(len(monthly_stats)).reshape(-1, 1)
y = monthly_stats['median_salary'].values
trend_model = LinearRegression().fit(X, y)
trendline = trend_model.predict(X)

# --- Plotting ---
fig = go.Figure()

# Actual Median Salary
fig.add_trace(go.Scatter(
    x=monthly_stats.index,
    y=monthly_stats['median_salary'],
    mode='lines+markers',
    name='Actual Median Salary',
    line=dict(color='#084594', width=3)
))

# Forecast Median Salary
fig.add_trace(go.Scatter(
    x=future_dates,
    y=salary_pred,
    mode='lines+markers',
    name='Forecast Median Salary (2025)',
    line=dict(color='#a6bddb', width=3, dash='dash')
))

# Confidence Interval
fig.add_trace(go.Scatter(
    x=future_dates.tolist() + future_dates[::-1].tolist(),
    y=salary_ci['lower median_salary'].tolist() + salary_ci['upper median_salary'][::-1].tolist(),
    fill='toself',
    fillcolor='rgba(8,69,148,0.12)',
    line=dict(color='rgba(255,255,255,0)'),
    hoverinfo="skip",
    showlegend=False
))

# Trendline
fig.add_trace(go.Scatter(
    x=monthly_stats.index,
    y=trendline,
    mode='lines',
    name='Trendline',
    line=dict(color='orange', width=2, dash='dot')
))

fig.update_layout(
    title='SARIMA Forecast for Data Scientist Salary in 2025',
    xaxis_title='Date',
    yaxis_title='Median Salary (EUR)',
    height=500,
    width=1000,
    template='plotly_white',
    margin=dict(t=60, b=40, l=60, r=40),
    legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1)
)

fig.show()

## Predict the most in-demand skills for Data Scientist in 2025

In [ ]:
# 1. Prepare the data: explode job_skills and group by month and skill
df_skills = df_filtered.dropna(subset=['job_skills', 'job_posted_date']).copy()
df_skills['job_posted_date'] = pd.to_datetime(df_skills['job_posted_date'])
df_skills['year'] = df_skills['job_posted_date'].dt.year
df_skills['month'] = df_skills['job_posted_date'].dt.month
df_skills = df_skills.explode('job_skills')

# 2. Get top N skills by total count (to focus on relevant skills)
top_n = 10
top_skills = (
    df_skills['job_skills']
    .value_counts()
    .head(top_n)
    .index.tolist()
)

# 3. Build a time series for each top skill
skill_forecasts = {}
future_months = pd.date_range(
    df_skills['job_posted_date'].max() + pd.offsets.MonthBegin(1),
    periods=12, freq='MS'
)

for skill in top_skills:
    # Monthly count of postings mentioning the skill
    skill_ts = (
        df_skills[df_skills['job_skills'] == skill]
        .groupby(['year', 'month'])
        .size()
        .reset_index(name='count')
    )
    skill_ts['date'] = pd.to_datetime(skill_ts[['year', 'month']].assign(day=1))
    # Aggregate by date to ensure uniqueness (fixes duplicate labels)
    skill_ts = skill_ts.groupby('date')['count'].sum().sort_index()
    # Fill missing months
    all_months = pd.date_range(skill_ts.index.min(), skill_ts.index.max(), freq='MS')
    skill_ts = skill_ts.reindex(all_months, fill_value=0)
    
    # SARIMA forecast
    try:
        model = SARIMAX(skill_ts, order=(1,1,1), seasonal_order=(1,1,1,12), enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False)
        forecast = results.get_forecast(steps=12)
        pred_mean = forecast.predicted_mean
        skill_forecasts[skill] = pred_mean.sum()  # total forecasted mentions in 2025
    except Exception as e:
        skill_forecasts[skill] = 0  # fallback if model fails

# 4. Rank skills by predicted demand in 2025
predicted_skills_2025 = sorted(skill_forecasts.items(), key=lambda x: x[1], reverse=True)

# Prepare data for plotting
skills = [s for s, _ in predicted_skills_2025]
demands = [max(0, d) for _, d in predicted_skills_2025]  # Clip negatives to zero

fig = go.Figure()

fig.add_trace(go.Bar(
    x=demands[::-1],
    y=skills[::-1],
    orientation='h',
    marker=dict(color=demands[::-1], colorscale='Blues'),
    text=[int(d) for d in demands[::-1]],
    textposition='auto',
    name='Forecasted Demand'
))

fig.update_layout(
    title='Top 10 Most In-Demand Data Scientist Skills (Forecast for 2025)',
    xaxis_title='Forecasted Number of Job Postings (2025)',
    yaxis_title='Skill',
    height=500,
    width=1000,
    template='plotly_white',
    margin=dict(t=60, b=40, l=60, r=40),
    showlegend=False
)

fig.show()

### Summary:
TBD